In [1]:
! pip install mlflow boto3 awscli optuna imbalanced-learn

In [2]:
import mlflow
mlflow.set_tracking_uri("http://98.84.31.179:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='s3://dsmp-mlflow-bucket/287048950003727427', creation_time=1734272062530, experiment_id='287048950003727427', last_update_time=1734272062530, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

C:\Users\kunkaush1\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Multinomial Naive Bayes

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: BOW vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_BOW_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2024-12-15 20:20:05,740] A new study created in memory with name: no-name-5586a31f-dc4f-4100-9162-789a05d2e373
[I 2024-12-15 20:20:05,955] Trial 0 finished with value: 0.6331642358909322 and parameters: {'alpha': 0.0025496713035951125}. Best is trial 0 with value: 0.6331642358909322.
[I 2024-12-15 20:20:05,973] Trial 1 finished with value: 0.6330585499894315 and parameters: {'alpha': 0.09902629190469188}. Best is trial 0 with value: 0.6331642358909322.
[I 2024-12-15 20:20:05,989] Trial 2 finished with value: 0.6329528640879307 and parameters: {'alpha': 0.1555093409583344}. Best is trial 0 with value: 0.6331642358909322.
[I 2024-12-15 20:20:06,005] Trial 3 finished with value: 0.6330585499894315 and parameters: {'alpha': 0.01050852783545269}. Best is trial 0 with value: 0.6331642358909322.
[I 2024-12-15 20:20:06,022] Trial 4 finished with value: 0.6330585499894315 and parameters: {'alpha': 0.01912897140427212}. Best is trial 0 with value: 0.6331642358909322.
[I 2024-12-15 20:20:06,03

🏃 View run MultinomialNB_SMOTE_BOW_Trigrams at: http://98.84.31.179:5000/#/experiments/287048950003727427/runs/c429877bd441455584a18cffc30135f6
🧪 View experiment at: http://98.84.31.179:5000/#/experiments/287048950003727427
